In [1]:
import pandas as pd
import numpy  as np
import sqlite3
import streamlit as st
import seaborn as sns
import matplotlib.pyplot as px
import plotly as plty

# Conectar a la base de datos SQLite
conn = sqlite3.connect(r"C:\Users\maria\Documents\UCV MARY\EECA\SEMESTRE 2024-1\SEMESTRE II\COMPUTACIÓN II\TRABAJO FINAL\SALUD MENTAL EN LA INDUSTRIA TECNOLÓGICA 1.sqlite")

cur = conn.cursor()

In [2]:
# Consultar las tablas en la base de datos
consulta_ntablas = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"


# Mostrar las entidades
print("Entidades en la base de datos:")
entidades = pd.read_sql_query(sql = consulta_ntablas, con = conn)
entidades

Entidades en la base de datos:


,name
0,Answer
1,Question
2,Respuestas
3,Survey


In [3]:
# Crear consulta para convertir en DF
consulta = "SELECT * FROM Respuestas;"

df_Respuestas = pd.read_sql_query(sql = consulta, con = conn)

conn.close()

In [4]:
# Filtrado del DF por desde 2016-2019 y Estados Unidos como país para realizar la Investigación
User_ID = df_Respuestas[(df_Respuestas['QuestionID'] == 3) & (df_Respuestas['AnswerText'] == 'Estados Unidos')]['UserID'].unique()

df_Investigacion1 = df_Respuestas[(df_Respuestas['UserID'].isin(User_ID)) & (df_Respuestas['SurveyID'] == 2016)]

df_Investigacion2 = df_Respuestas[(df_Respuestas['UserID'].isin(User_ID)) & (df_Respuestas['SurveyID'] == 2017)]

df_Investigacion3 = df_Respuestas[(df_Respuestas['UserID'].isin(User_ID)) & (df_Respuestas['SurveyID'] == 2018)]

df_Investigacion4 = df_Respuestas[(df_Respuestas['UserID'].isin(User_ID)) & (df_Respuestas['SurveyID'] == 2019)]

In [5]:
# Reestructuración del DF para mejor uso de Pandas
df_Investigacion1 = df_Investigacion1.groupby(['UserID', 'QuestionID'])['AnswerText'].first().reset_index()

df_Investigacion2 = df_Investigacion2.groupby(['UserID', 'QuestionID'])['AnswerText'].first().reset_index()

df_Investigacion3 = df_Investigacion3.groupby(['UserID', 'QuestionID'])['AnswerText'].first().reset_index()

df_Investigacion4 = df_Investigacion4.groupby(['UserID', 'QuestionID'])['AnswerText'].first().reset_index()

df_Investigacion1 = df_Investigacion1.pivot_table(index='UserID', columns='QuestionID', values='AnswerText', aggfunc='first')

df_Investigacion2 = df_Investigacion2.pivot_table(index='UserID', columns='QuestionID', values='AnswerText', aggfunc='first')

df_Investigacion3 = df_Investigacion3.pivot_table(index='UserID', columns='QuestionID', values='AnswerText', aggfunc='first')

df_Investigacion4 = df_Investigacion4.pivot_table(index='UserID', columns='QuestionID', values='AnswerText', aggfunc='first')

In [6]:
# Renombrar las variables 2016
print(df_Investigacion1.columns)

df_Investigacion1.columns = ['Edad', 'Género', 'País de Residencia', 'Ciudad de Residencia (EEUU)', 'Trabajo Autónomo', 'Historial Familiar', 'Búsqueda de Tratamiento con un Profesional', 'Cantidad de Empleados en la Empresa', 'Beneficios Dentro de Seguro Médico', 'Problemas de Salud Mental en Entrevista Laboral', 'Conocimiento de las Opciones de Cobertura por la Empresa Actual', 'Recursos de la Empresa Para Conocer sobre la Salud Mental y Canales de Ayuda', 'Facilidad de Consulta de Baja Médica en el Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con Compañeros de Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con un Director/Supervisor', 'Cobertura de Salud Mental en Seguro Médico', 'Beneficios de Salud Mental de Empleos Anteriores', 'Conocimiento de las Opciones de Ayuda por el Empleo Anterior', 'Protección de Anonimato Si Se Toma Ventaja de los Recursos de Tratamiento en Empleos Anteriores', 'Disposición de Hablar de Salud Mental con un Supervisor', 'Disposición para Hablar Sobre Enfermedad Mental con Familia/Amigos', 'Seguridad de Revelar Enfermedad Mental Debido a Comentario Sobre Salud Mental de Otra Persona', 'Situación Mental Actual', '¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?', 'Nivel de Interferencia en el Trabajo Cuando Se Está Bajo Tratamiento', 'Nivel de Interferencia en el Trabajo Cuando No Se Está Bajo Tratamiento', 'Disposición a Revelar Enfermedad Mental a Empleados/Compañeros', 'Productividad Afectada Por Enfermedad Mental', 'Porcentaje de Tiempo Afectada Por Enfermedad Mental', '¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?', '¿Cree que los Miembros de su Equipo/Compañeros de Trabajo le Verían de Forma más Negativa Si Supieran que Padece una Enfermedad Mental?', 'Enfermedades Mentales Diagnosticadas', 'Enfermedades Mentales No Diagnosticadas', 'Puesto de Trabajo', 'Trabajo Remoto']

# Reemplazar los NaN
df_Investigacion1 = df_Investigacion1.fillna('No respondió')

Index([  1,   2,   3,   4,   5,   6,   7,   8,  10,  12,  14,  16,  17,  18,
        19,  20,  23,  24,  27,  28,  30,  31,  33,  34,  48,  49,  53,  54,
        55,  56, 114, 115, 116, 117, 118],
      dtype='int64', name='QuestionID')


In [7]:
# Renombrar las variables 2017
print(df_Investigacion2.columns)

df_Investigacion2.columns = ['Edad', 'Género', 'País de Residencia', 'Ciudad de Residencia (EEUU)', 'Trabajo Autónomo', 'Historial Familiar', 'Búsqueda de Tratamiento con un Profesional', 'Cantidad de Empleados en la Empresa', 'Beneficios Dentro de Seguro Médico', 'Problemas de Salud Mental en Entrevista Laboral', 'Conocimiento de las Opciones de Cobertura por la Empresa Actual', 'Recursos de la Empresa Para Conocer sobre la Salud Mental y Canales de Ayuda', 'Facilidad de Consulta de Baja Médica en el Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con Compañeros de Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con un Director/Supervisor', 'Cobertura de Salud Mental en Seguro Médico', 'Beneficios de Salud Mental de Empleos Anteriores', 'Conocimiento de las Opciones de Ayuda por el Empleo Anterior', 'Protección de Anonimato Si Se Toma Ventaja de los Recursos de Tratamiento en Empleos Anteriores', 'Disposición de Hablar de Salud Mental con un Supervisor', 'Disposición para Hablar Sobre Enfermedad Mental con Familia/Amigos', 'Seguridad de Revelar Enfermedad Mental Debido a Comentario Sobre Salud Mental de Otra Persona', 'Situación Mental Actual', '¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?', 'Nivel de Interferencia en el Trabajo Cuando Se Está Bajo Tratamiento', 'Nivel de Interferencia en el Trabajo Cuando No Se Está Bajo Tratamiento', 'Disposición a Revelar Enfermedad Mental a Empleados/Compañeros', 'Productividad Afectada Por Enfermedad Mental', 'Porcentaje de Tiempo Afectada Por Enfermedad Mental', '¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?', 'Importancia a la Salud Mental (Empresa)', '¿Cómo Afectó la Relación con Alguien de tu Trabajo el Haberle Contado sobre tu Enfermedad Mental?', '¿Te Identificas Abiertamente en el Trabajo como una Persona con una Enfermedad Mental?', '¿Tu Carrera Se Ha Visto Afectada por Tener una Enfermedad Mental?', 'Si Supieran Que Padeces un Trastorno Mental, ¿Cómo Crees Que Reaccionarían los Miembros de tu Equipo/Compañeros de Trabajo?', '¿Has Observado o Experimentado una Respuesta Positiva De Compañeros/Jefes al Saber que Tienes una Enfermedad Mental?', '¿En qué Medida Cree que el Sector Tecnológico Apoya a los Empleados con Enfermedades Mentales?', 'Raza']

# Reemplazar los NaN
df_Investigacion2 = df_Investigacion2.fillna('No respondió')

Index([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 12, 14, 16, 17, 18, 19, 20, 23, 24,
       27, 28, 30, 31, 33, 34, 48, 49, 53, 54, 55, 56, 65, 67, 78, 79, 81, 83,
       85, 89],
      dtype='int64', name='QuestionID')


In [8]:
# Renombrar las variables 2018
print(df_Investigacion3.columns)

df_Investigacion3.columns = ['Edad', 'Género', 'País de Residencia', 'Ciudad de Residencia (EEUU)', 'Trabajo Autónomo', 'Historial Familiar', 'Búsqueda de Tratamiento con un Profesional', 'Cantidad de Empleados en la Empresa', 'Beneficios Dentro de Seguro Médico', 'Problemas de Salud Mental en Entrevista Laboral', 'Conocimiento de las Opciones de Cobertura por la Empresa Actual', 'Recursos de la Empresa Para Conocer sobre la Salud Mental y Canales de Ayuda', 'Facilidad de Consulta de Baja Médica en el Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con Compañeros de Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con un Director/Supervisor', 'Cobertura de Salud Mental en Seguro Médico', 'Beneficios de Salud Mental de Empleos Anteriores', 'Conocimiento de las Opciones de Ayuda por el Empleo Anterior', 'Protección de Anonimato Si Se Toma Ventaja de los Recursos de Tratamiento en Empleos Anteriores', 'Disposición de Hablar de Salud Mental con un Supervisor', 'Disposición para Hablar Sobre Enfermedad Mental con Familia/Amigos', 'Seguridad de Revelar Enfermedad Mental Debido a Comentario Sobre Salud Mental de Otra Persona', 'Situación Mental Actual', '¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?', 'Nivel de Interferencia en el Trabajo Cuando Se Está Bajo Tratamiento', 'Nivel de Interferencia en el Trabajo Cuando No Se Está Bajo Tratamiento', 'Disposición a Revelar Enfermedad Mental a Empleados/Compañeros', 'Productividad Afectada Por Enfermedad Mental', 'Porcentaje de Tiempo Afectada Por Enfermedad Mental', '¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?', 'Importancia a la Salud Mental (Empresa)', '¿Cómo Afectó la Relación con Alguien de tu Trabajo el Haberle Contado sobre tu Enfermedad Mental?', '¿Te Identificas Abiertamente en el Trabajo como una Persona con una Enfermedad Mental?', '¿Tu Carrera Se Ha Visto Afectada por Tener una Enfermedad Mental?', 'Si Supieran Que Padeces un Trastorno Mental, ¿Cómo Crees Que Reaccionarían los Miembros de tu Equipo/Compañeros de Trabajo?', '¿Has Observado o Experimentado una Respuesta Positiva De Compañeros/Jefes al Saber que Tienes una Enfermedad Mental?', '¿En qué Medida Cree que el Sector Tecnológico Apoya a los Empleados con Enfermedades Mentales?', 'Raza']

# Reemplazar los NaN
df_Investigacion3 = df_Investigacion3.fillna('No respondió')

Index([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 12, 14, 16, 17, 18, 19, 20, 23, 24,
       27, 28, 30, 31, 33, 34, 48, 49, 53, 54, 55, 56, 65, 67, 78, 79, 81, 83,
       85, 89],
      dtype='int64', name='QuestionID')


In [9]:
# Renombrar las variables 2019
print(df_Investigacion4.columns)

df_Investigacion4.columns = ['Edad', 'Género', 'País de Residencia', 'Ciudad de Residencia (EEUU)', 'Trabajo Autónomo', 'Historial Familiar', 'Búsqueda de Tratamiento con un Profesional', 'Cantidad de Empleados en la Empresa', 'Beneficios Dentro de Seguro Médico', 'Problemas de Salud Mental en Entrevista Laboral', 'Conocimiento de las Opciones de Cobertura por la Empresa Actual', 'Recursos de la Empresa Para Conocer sobre la Salud Mental y Canales de Ayuda', 'Facilidad de Consulta de Baja Médica en el Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con Compañeros de Trabajo', 'Comodidad para Hablar de una Enfermedad Mental con un Director/Supervisor', 'Cobertura de Salud Mental en Seguro Médico', 'Beneficios de Salud Mental de Empleos Anteriores', 'Conocimiento de las Opciones de Ayuda por el Empleo Anterior', 'Protección de Anonimato Si Se Toma Ventaja de los Recursos de Tratamiento en Empleos Anteriores', 'Disposición de Hablar de Salud Mental con un Supervisor', 'Disposición para Hablar Sobre Enfermedad Mental con Familia/Amigos', 'Seguridad de Revelar Enfermedad Mental Debido a Comentario Sobre Salud Mental de Otra Persona', 'Situación Mental Actual', '¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?', 'Nivel de Interferencia en el Trabajo Cuando Se Está Bajo Tratamiento', 'Nivel de Interferencia en el Trabajo Cuando No Se Está Bajo Tratamiento', 'Disposición a Revelar Enfermedad Mental a Empleados/Compañeros', 'Productividad Afectada Por Enfermedad Mental', 'Porcentaje de Tiempo Afectada Por Enfermedad Mental', '¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?', 'Importancia a la Salud Mental (Empresa)', '¿Cómo Afectó la Relación con Alguien de tu Trabajo el Haberle Contado sobre tu Enfermedad Mental?', '¿Te Identificas Abiertamente en el Trabajo como una Persona con una Enfermedad Mental?', '¿Tu Carrera Se Ha Visto Afectada por Tener una Enfermedad Mental?', 'Si Supieran Que Padeces un Trastorno Mental, ¿Cómo Crees Que Reaccionarían los Miembros de tu Equipo/Compañeros de Trabajo?', '¿Has Observado o Experimentado una Respuesta Positiva De Compañeros/Jefes al Saber que Tienes una Enfermedad Mental?', '¿En qué Medida Cree que el Sector Tecnológico Apoya a los Empleados con Enfermedades Mentales?', 'Raza']

# Reemplazar los NaN
df_Investigacion4 = df_Investigacion4.fillna('No respondió')

Index([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 12, 14, 16, 17, 18, 19, 20, 23, 24,
       27, 28, 30, 31, 33, 34, 48, 49, 53, 54, 55, 56, 65, 67, 78, 79, 81, 83,
       85, 89],
      dtype='int64', name='QuestionID')


In [13]:
# Guardar el resultado en un archivo .csv

df_Investigacion1.to_csv('Investigacion1.csv', index=False)
df_Investigacion2.to_csv('Investigacion2.csv', index=False)
df_Investigacion3.to_csv('Investigacion3.csv', index=False)
df_Investigacion4.to_csv('Investigacion4.csv', index=False)

# Leer archivo Investigacion1.csv
df_Investigacion1 = pd.read_csv('Investigacion1.csv')

# Leer archivo Investigacion2.csv
df_Investigacion2 = pd.read_csv('Investigacion2.csv')

# Leer archivo Investigacion3.csv
df_Investigacion3 = pd.read_csv('Investigacion3.csv')

# Leer archivo Investigacion4.csv
df_Investigacion4 = pd.read_csv('Investigacion4.csv')

In [15]:
# Título de la aplicación
st.title("Problemáticas y Estigmas de las Enfermedades Mentales en la Industria Tecnológica Estadounidense 2016-2019")

# Crea un menú de pestañas
tab1, tab2 = st.tabs(["Página 1", "Página 2"])

# Contenido de la página 1
with tab1:
    st.header("Datos")
    st.write("Información de la Data Suministrada.")

# Carga los DataFrames desde los archivos.csv
df_2016 = pd.read_csv('Investigacion1.csv')
df_2017 = pd.read_csv('Investigacion2.csv')
df_2018 = pd.read_csv('Investigacion3.csv')
df_2019 = pd.read_csv('Investigacion4.csv')

# Crea un selectbox con las opciones
option = st.selectbox('Año de Encuesta:', ['2016', '2017', '2018', '2019'])

# Muestra el DataFrame correspondiente según la opción seleccionada
if option == '2016':
    st.dataframe(df_2016)
elif option == '2017':
    st.dataframe(df_2017)
elif option == '2018':
    st.dataframe(df_2018)
elif option == '2019':
    st.dataframe(df_2019)

st.write("""
## Introducción

El conjunto de datos **Iris** es uno de los conjuntos de datos más conocidos en la comunidad de ciencia de datos. Este conjunto de datos fue introducido por el biólogo y estadístico británico Ronald A. Fisher en su artículo de 1936 "The use of multiple measurements in taxonomic problems". El conjunto de datos contiene 150 observaciones de iris con cuatro características: longitud del sépalo, anchura del sépalo, longitud del pétalo y anchura del pétalo. Además, cada observación pertenece a una de las tres especies de iris: Iris setosa, Iris versicolor o Iris virginica.

En esta aplicación, exploraremos el conjunto de datos Iris mediante gráficos y estadísticas descriptivas. A continuación, se muestra una tabla con las primeras filas del conjunto de datos.
""")

In [18]:
# PREGUNTA #1
#¿Cuántas personas en la industria tecnológica tienen una enfermedad mental diagnosticada y, dentro de este grupo, existe algún historial familiar dentro de este ámbito?

Pregunta1 = pd.crosstab(df_Investigacion1['¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?'], 
                        df_Investigacion1['Historial Familiar'], 
                        margins=True, 
                        margins_name='Total')

Pregunta1.columns.name = 'Historial Familiar'
Pregunta1.index.name = '¿Alguna Vez Has Sido Diagnosticado con una Enfermedad Mental?'

st.write("Tabla de Contingencia")
st.write(Pregunta1)


In [19]:
#PREGUNTA 3
#¿Cuál es el porcentaje de contratación que tienen las empresas estadounidenses a la hora de emplear personas con enfermedades mentales? ¿Estas ofrecen algún beneficio o algún tipo de convenio de trabajo?
# Filter rows where the question is 'situacion mental actual'
filtered_df = df_Investigacion[df_Investigacion['Situación Mental Actual'] == 'Si']

# Group by userid to count the number of people in the company
grouped_df = filtered_df.groupby('UserID').size().reset_index(name='Cantidad de Empleados en la Empresa')

# Join this result with the original dataframe df_investigacion using the userid as the key
result_df = df_Investigacion.merge(grouped_df, on='UserID', how='left')

new_df = result_df[['Situación Mental Actual', 'Cantidad de Empleados en la Empresa']].copy()


KeyError: "['Cantidad de Empleados en la Empresa'] not in index"

In [13]:
#PREGUNTA 6

# Agregar un índice único a cada dataframe
df_Investigacion = df_Investigacion.reset_index()
df_Investigacion = df_Investigacion.rename(columns={'index': 'UserID'})

# Asignar las columnas que deseas unir a una lista
columns_to_join = ["Situación Mental Actual", "Productividad Afectada Por Enfermedad Mental", "Porcentaje de Tiempo Afectada por Enfermedad Mental"]

# Realizar el join utilizando la columna UserID como clave
joined_df = df_Investigacion.set_index("UserID")[columns_to_join].join(df_Investigacion.set_index("UserID")[columns_to_join], on="UserID")

# Resetear el índice para que UserID sea una columna nuevamente
joined_df.reset_index(inplace=True)

KeyError: "['Porcentaje de Tiempo Afectada por Enfermedad Mental'] not in index"

In [14]:
# ¿Si Expresas que posees una Enfermedad Mental crees que en tu ambiente de Trabajo seria tomado de Mala manera?
Pregunta_5 = df_Investigacion['¿Has Observado o Experimentado una Respuesta Insolidaria o Mal Gestionada a un Problema de Salud Mental en tu Lugar de Trabajo Actual o Anterior?'].value_counts().to_frame('Posee Enfermedad Mental Diagnosticada')

Pregunta_5 = Pregunta_5.rename(columns={'count': 'Posee Enfermedad Mental Diagnosticada'})

print(Pregunta_5)

                                                    Posee Enfermedad Mental Diagnosticada
¿Has Observado o Experimentado una Respuesta In...                                       
No                                                                                    341
Tal vez/No estoy seguro(a)                                                            215
Si, lo he visto                                                                       155
Si, me ha pasado                                                                       89
No respondió                                                                           39


In [15]:
import matplotlib.colors as mcolors
# Create an array of colors with a gradient
colors = np.array(['#6495ED', '#87CEEB', '#ADD8E6', '#C6E2B5', '#D9F0D9'])

# Sort the values and indices of Pregunta_5 in descending order
sorted_indices = Pregunta_5['Posee Enfermedad Mental Diagnosticada'].sort_values(ascending=False).index
sorted_values = Pregunta_5['Posee Enfermedad Mental Diagnosticada'].sort_values(ascending=False)

# Reverse the order of the sorted indices and values
sorted_indices = sorted_indices[::-1]
sorted_values = sorted_values[::-1]

# Create the colors gradient (use the sorted indices to select the colors)
colors_grad = [colors[i % len(colors)] for i in range(len(sorted_values))]

# Create the horizontal bar chart
plt.barh(sorted_indices, sorted_values, color=colors_grad)

# Add labels, title, etc.
plt.xlabel('Cantidad')
plt.ylabel('Reacción al confesar tener una Enfermedad Mnetal')
plt.title('Posee Enfermedad Mental Diagnosticada')

# Show the chart
plt.show()

C:\Users\maria\AppData\Local\Temp\ipykernel_9460\2807150385.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [16]:
# Pregunta 2
# ¿Cuáles son las 5 enfermedades más comunes de los encuestados según el estado donde residen? ¿Cambiaría este ranking si se toma en cuenta la edad y el genero? 
pregunta_2 = df_Investigacion[df_Investigacion['Enfermedades Mentales Diagnosticadas']!= 'No respondió'].groupby('Ciudad de Residencia (EEUU)')['Enfermedades Mentales Diagnosticadas'].value_counts().nlargest(5).reset_index(name='Frecuencia')

pregunta_2 = pregunta_2.loc[:, ['Enfermedades Mentales Diagnosticadas', 'Ciudad de Residencia (EEUU)', 'Frecuencia']]  # Reordenar columnas

pregunta_2.columns = ['Enfermedades Mentales Diagnosticadas', 'Ciudad de Residencia (EEUU)', 'Cantidad de personas']  # Renombrar columnas

print(pregunta_2.to_string(header=True, index=False))

                               Enfermedades Mentales Diagnosticadas Ciudad de Residencia (EEUU)  Cantidad de personas
           Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)                  California                    34
Trastorno del Estado del Ánimo (Depresión, Trastorno Bipolar, etc.)                  California                    19
           Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)                    Illinois                    18
           Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)                 Pensilvania                    16
           Trastorno de Ansiedad (Genralizado, Social, Fobia, etc.)                  Nueva York                    13


In [17]:
# Parte 3, pregunta 2
Pregunta2_3 = df_Investigacion.groupby(['Enfermedades Mentales Diagnosticadas', 'Género']).size().reset_index(name='Cantidad de personas')

Pregunta2_3 = Pregunta2_3[Pregunta2_3['Género']!= 'No respondió']  # Eliminar "No respondió" de la columna "Género"

top_5_enfermedades = Pregunta2_3.groupby('Enfermedades Mentales Diagnosticadas')['Cantidad de personas'].sum().nlargest(6).index

Pregunta2_3 = Pregunta2_3[Pregunta2_3['Enfermedades Mentales Diagnosticadas'].isin(top_5_enfermedades)]

Pregunta2_3 = Pregunta2_3[Pregunta2_3['Enfermedades Mentales Diagnosticadas']!= 'No respondió']

Pregunta2_3 = Pregunta2_3.pivot_table(index='Enfermedades Mentales Diagnosticadas', columns='Género', values='Cantidad de personas', fill_value=0)

print(Pregunta2_3)

Género                                              Femenino  Masculino  Otro
Enfermedades Mentales Diagnosticadas                                         
Trastorno Obsesivo-Compulsivo                            0.0        3.0   0.0
Trastorno de Ansiedad (Genralizado, Social, Fob...      90.0      130.0   7.0
Trastorno de Estrés Post-Traumático                      1.0        6.0   0.0
Trastorno del Estado del Ánimo (Depresión, Tras...      45.0       66.0   5.0
Trastorno por Déficit de Atención e Hiperactividad       2.0       16.0   1.0


In [18]:
# Parte 2, Pregunta 2
df_Investigacionp2=df_Investigacion
# Convertir la columna Edad a tipo numérico, convirtiendo valores no numéricos a NaN
df_Investigacionp2['Edad'] = pd.to_numeric(df_Investigacionp2['Edad'], errors='coerce')

# Eliminar filas con valores NaN en la columna Edad
df_Investigacionp2 = df_Investigacionp2.dropna(subset=['Edad'])

# Convertir la columna Edad a tipo entero
df_Investigacionp2['Edad'] = df_Investigacionp2['Edad'].astype(int)

# Convertir la columna Enfermedades Mentales Diagnosticadas a tipo string
df_Investigacionp2['Enfermedades Mentales Diagnosticadas'] = df_Investigacionp2['Enfermedades Mentales Diagnosticadas'].astype(str)

# Crear una columna Edad_binned con los rangos de edad
df_Investigacionp2['Edad'] = pd.cut(df_Investigacionp2['Edad'], bins=[18, 25, 32, 38, 44, 50, 56, 66], 
                                 labels=['19-25', '26-32', '33-38', '39-44', '45-50', '51-56', '57-66'],
                                 include_lowest=True)

# Eliminar filas con "No respondio" en la columna Enfermedades Mentales Diagnosticadas
df_Investigacionp2 = df_Investigacionp2[~df_Investigacionp2['Enfermedades Mentales Diagnosticadas'].isin(['No respondió'])]

# Agrupar por Edad y Enfermedades Mentales Diagnosticadas, y contar la frecuencia
pregunta2_2 = df_Investigacionp2.groupby(['Edad', 'Enfermedades Mentales Diagnosticadas']).size().reset_index(name='Cantidad de personas')

# Seleccionar la enfermedad mental más frecuente para cada rango de edad
pregunta2_2 = pregunta2_2.loc[pregunta2_2.groupby('Edad')['Cantidad de personas'].idxmax()]

# Reordenar las columnas
pregunta2_2 = pregunta2_2[['Enfermedades Mentales Diagnosticadas', 'Edad', 'Cantidad de personas']]

# Mostrar la tabla con la enfermedad mental más frecuente para cada rango de edad
print(pregunta2_2)

                 Enfermedades Mentales Diagnosticadas   Edad  \
8   Trastorno de Ansiedad (Genralizado, Social, Fo...  19-25   
22  Trastorno de Ansiedad (Genralizado, Social, Fo...  26-32   
36  Trastorno de Ansiedad (Genralizado, Social, Fo...  33-38   
50  Trastorno de Ansiedad (Genralizado, Social, Fo...  39-44   
64  Trastorno de Ansiedad (Genralizado, Social, Fo...  45-50   
78  Trastorno de Ansiedad (Genralizado, Social, Fo...  51-56   
92  Trastorno de Ansiedad (Genralizado, Social, Fo...  57-66   

    Cantidad de personas  
8                     37  
22                    75  
36                    65  
50                    28  
64                    14  
78                     5  
92                     4  


C:\Users\maria\AppData\Local\Temp\ipykernel_9460\3695608321.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Investigacionp2['Edad'] = df_Investigacionp2['Edad'].astype(int)
C:\Users\maria\AppData\Local\Temp\ipykernel_9460\3695608321.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Investigacionp2['Enfermedades Mentales Diagnosticadas'] = df_Investigacionp2['Enfermedades Mentales Diagnosticadas'].astype(str)
C:\Users\maria\AppData\Local\Temp\ipykernel_9460\3695608321.py:16: SettingWithCopyWarn